In [95]:
import pandas as pd
import re

pd.set_option('display.max_columns', None)

In [96]:
df = pd.read_csv('all_records.csv')

In [97]:
df.head()

,Unnamed: 0,Animal ID,Name_intake,DateTime_intake,MonthYear_intake,Found_Location,Intake_Type,IntakeCondition,Animal_Type_intake,Sex,Age,Breed_intake,Color_intake,Name_outcome,DateTime_outcome,MonthYear_outcome,Outcome_Type,Outcome_Subtype,Sex_upon_Outcome,Age_upon_Outcome,gender_intake,gender_outcome,fixed_intake,fixed_outcome,fixed_changed,Age_Bucket,retriever,shepherd,beagle,terrier,boxer,poodle,rottweiler,dachshund,chihuahua,pit bull,DateTime_length,Days_length
0,0,A730601,NaN,2016-07-07 12:11:00,07/07/2016 12:11:00 PM,1109 Shady Ln in Austin (TX),Stray,Normal,Cat,Intact Male,7 months,Domestic Shorthair Mix,Blue Tabby,NaN,2016-07-08 09:00:00,07/08/2016 09:00:00 AM,Transfer,SCRP,Neutered Male,7 months,Male,Male,Intact,Neutered,1,7-12 months,0,0,0,0,0,0,0,0,0,0,0 days 20:49:00.000000000,0-7 days
1,1,A683644,*Zoey,2014-07-13 11:02:00,07/13/2014 11:02:00 AM,Austin (TX),Owner Surrender,Nursing,Dog,Intact Female,4 weeks,Border Collie Mix,Brown/White,*Zoey,2014-11-06 10:06:00,11/06/2014 10:06:00 AM,Adoption,Foster,Spayed Female,4 months,Female,Female,Intact,Spayed,1,1-6 weeks,0,0,0,0,0,0,0,0,0,0,115 days 23:04:00.000000000,12 weeks - 6 months
2,2,A676515,Rico,2014-04-11 08:45:00,04/11/2014 08:45:00 AM,615 E. Wonsley in Austin (TX),Stray,Normal,Dog,Intact Male,2 months,Pit Bull Mix,White/Brown,Rico,2014-04-14 18:38:00,04/14/2014 06:38:00 PM,Return to Owner,NaN,Neutered Male,3 months,Male,Male,Intact,Neutered,1,1-6 months,0,0,0,0,0,0,0,0,0,1,3 days 09:53:00.000000000,0-7 days
3,3,A742953,NaN,2017-01-31 13:30:00,01/31/2017 01:30:00 PM,S Hwy 183 And Thompson Lane in Austin (TX),Stray,Normal,Dog,Intact Male,2 years,Saluki,Sable/Cream,NaN,2017-02-04 14:17:00,02/04/2017 02:17:00 PM,Transfer,Partner,Intact Male,2 years,Male,Male,Intact,Intact,0,1-3 years,0,0,0,0,0,0,0,0,0,0,4 days 00:47:00.000000000,0-7 days
4,4,A679549,*Gilbert,2014-05-22 15:43:00,05/22/2014 03:43:00 PM,124 W Anderson in Austin (TX),Stray,Normal,Cat,Intact Male,1 month,Domestic Shorthair Mix,Black/White,*Gilbert,2014-06-16 13:54:00,06/16/2014 01:54:00 PM,Transfer,Partner,Neutered Male,2 months,Male,Male,Intact,Neutered,1,1-6 months,0,0,0,0,0,0,0,0,0,0,24 days 22:11:00.000000000,3-6 weeks


In [98]:
# Dataframe info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76977 entries, 0 to 76976
Data columns (total 38 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          76977 non-null  int64 
 1   Animal ID           76977 non-null  object
 2   Name_intake         57493 non-null  object
 3   DateTime_intake     76977 non-null  object
 4   MonthYear_intake    76977 non-null  object
 5   Found_Location      76977 non-null  object
 6   Intake_Type         76977 non-null  object
 7   IntakeCondition     76977 non-null  object
 8   Animal_Type_intake  76977 non-null  object
 9   Sex                 76976 non-null  object
 10  Age                 76977 non-null  object
 11  Breed_intake        76977 non-null  object
 12  Color_intake        76977 non-null  object
 13  Name_outcome        57493 non-null  object
 14  DateTime_outcome    76977 non-null  object
 15  MonthYear_outcome   76977 non-null  object
 16  Outcome_Type        76

In [99]:
# Dataframe null values
df.isnull().sum()

Unnamed: 0                0
Animal ID                 0
Name_intake           19484
DateTime_intake           0
MonthYear_intake          0
Found_Location            0
Intake_Type               0
IntakeCondition           0
Animal_Type_intake        0
Sex                       1
Age                       0
Breed_intake              0
Color_intake              0
Name_outcome          19484
DateTime_outcome          0
MonthYear_outcome         0
Outcome_Type              7
Outcome_Subtype       45254
Sex_upon_Outcome          4
Age_upon_Outcome         21
gender_intake          5608
gender_outcome         5611
fixed_intake              1
fixed_outcome             4
fixed_changed             0
Age_Bucket                0
retriever                 0
shepherd                  0
beagle                    0
terrier                   0
boxer                     0
poodle                    0
rottweiler                0
dachshund                 0
chihuahua                 0
pit bull            

In [100]:
# Drop duplicate and null value columns
df.drop(['Unnamed: 0', 'MonthYear_intake', 'MonthYear_outcome','Outcome_Subtype', 'Name_outcome', 'Days_length'], 
        axis=1, inplace=True)

In [101]:
# Change Name_Intake to Yes or No
df.Name_intake = df.Name_intake.apply(lambda x: 'No' if pd.isnull(x) else 'Yes')
df.Name_intake.value_counts()

Yes    57493
No     19484
Name: Name_intake, dtype: int64

In [102]:
# Drop rows with Null values
df = df.dropna()
df.isnull().sum()

Animal ID             0
Name_intake           0
DateTime_intake       0
Found_Location        0
Intake_Type           0
IntakeCondition       0
Animal_Type_intake    0
Sex                   0
Age                   0
Breed_intake          0
Color_intake          0
DateTime_outcome      0
Outcome_Type          0
Sex_upon_Outcome      0
Age_upon_Outcome      0
gender_intake         0
gender_outcome        0
fixed_intake          0
fixed_outcome         0
fixed_changed         0
Age_Bucket            0
retriever             0
shepherd              0
beagle                0
terrier               0
boxer                 0
poodle                0
rottweiler            0
dachshund             0
chihuahua             0
pit bull              0
DateTime_length       0
dtype: int64

In [103]:
df.Outcome_Type.value_counts()

Adoption           32360
Transfer           18375
Return to Owner    17356
Euthanasia          2768
Died                 390
Missing               50
Disposal              34
Rto-Adopt             23
Relocate               1
Name: Outcome_Type, dtype: int64

In [104]:
# Combine Died & Disposal Outcomes
df.loc[df.Outcome_Type == 'Disposal', 'Outcome_Type'] = 'Died'
df.Outcome_Type.value_counts()

Adoption           32360
Transfer           18375
Return to Owner    17356
Euthanasia          2768
Died                 424
Missing               50
Rto-Adopt             23
Relocate               1
Name: Outcome_Type, dtype: int64

In [105]:
# Combine Transfer & Relocate Outcomes
df.loc[df.Outcome_Type == 'Relocate', 'Outcome_Type'] = 'Transfer'
df.Outcome_Type.value_counts()

Adoption           32360
Transfer           18376
Return to Owner    17356
Euthanasia          2768
Died                 424
Missing               50
Rto-Adopt             23
Name: Outcome_Type, dtype: int64

In [106]:
# Drop Rto-Adopt and Missing Outcomes
df.drop(df[(df.Outcome_Type == 'Rto-Adopt') | (df.Outcome_Type == 'Missing')].index, inplace=True)
df.Outcome_Type.value_counts()

Adoption           32360
Transfer           18376
Return to Owner    17356
Euthanasia          2768
Died                 424
Name: Outcome_Type, dtype: int64

In [107]:
# Update DateTime_intake to datatime datatype
df.DateTime_intake = pd.to_datetime(df.DateTime_intake, format='%Y-%m-%d')
# Update DateTime_outcome to datatime datatype
df.DateTime_outcome = pd.to_datetime(df.DateTime_intake, format='%Y-%m-%d')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71284 entries, 0 to 76976
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Animal ID           71284 non-null  object        
 1   Name_intake         71284 non-null  object        
 2   DateTime_intake     71284 non-null  datetime64[ns]
 3   Found_Location      71284 non-null  object        
 4   Intake_Type         71284 non-null  object        
 5   IntakeCondition     71284 non-null  object        
 6   Animal_Type_intake  71284 non-null  object        
 7   Sex                 71284 non-null  object        
 8   Age                 71284 non-null  object        
 9   Breed_intake        71284 non-null  object        
 10  Color_intake        71284 non-null  object        
 11  DateTime_outcome    71284 non-null  datetime64[ns]
 12  Outcome_Type        71284 non-null  object        
 13  Sex_upon_Outcome    71284 non-null  object    

In [108]:
df.IntakeCondition.value_counts()

Normal      64686
Injured      2975
Sick         1867
Nursing      1211
Aged          296
Other         128
Feral          73
Pregnant       48
Name: IntakeCondition, dtype: int64

In [109]:
# Combine Aged and Feral with Other for IntakeCondition
df.loc[(df.IntakeCondition == 'Aged') | (df.IntakeCondition == 'Feral'), 'IntakeCondition'] = 'Other'

In [110]:
# Combine Injured and Sick for Intake Condition
df.loc[(df.IntakeCondition == 'Injured') | (df.IntakeCondition == 'Sick'), 'IntakeCondition'] = 'Medical'

In [111]:
# Combine Pregnant and Nursing for  IntakeCondition
df.loc[(df.IntakeCondition == 'Nursing') | (df.IntakeCondition == 'Pregnant'), 'IntakeCondition' ] = 'Maternity'
df.IntakeCondition.value_counts()

Normal       64686
Medical       4842
Maternity     1259
Other          497
Name: IntakeCondition, dtype: int64

In [112]:
df.Animal_Type_intake.value_counts()

Dog          47884
Cat          22815
Other          461
Bird           118
Livestock        6
Name: Animal_Type_intake, dtype: int64

In [113]:
# Combine Bird and Livestock with Other for Animal_Type_intake
df.loc[(df.Animal_Type_intake == 'Bird') | (df.Animal_Type_intake == 'Livestock'), 'Animal_Type_intake'] = 'Other'
df.Animal_Type_intake.value_counts()

Dog      47884
Cat      22815
Other      585
Name: Animal_Type_intake, dtype: int64

In [114]:
df.Sex.value_counts()

Intact Male      22323
Intact Female    20429
Neutered Male    15355
Spayed Female    13177
Name: Sex, dtype: int64

In [115]:
df.rename(columns={'Sex': 'Sex_Intake'}, inplace=True)

In [116]:
df.Sex_upon_Outcome.value_counts()

Neutered Male    29549
Spayed Female    25964
Intact Male       8129
Intact Female     7642
Name: Sex_upon_Outcome, dtype: int64

In [117]:
df.gender_outcome.value_counts()

Male      37678
Female    33606
Name: gender_outcome, dtype: int64

In [118]:
df.gender_intake.value_counts()

Male      37678
Female    33606
Name: gender_intake, dtype: int64

In [119]:
# Drop gender intake and outcome columns as no change and information in Sex columns
df.drop(['gender_outcome', 'gender_intake'], axis=1, inplace=True)

In [120]:
df.Age_Bucket.value_counts()

1-3 years           30850
1-6 months          16581
4-6 years            8656
7+ years             5432
1-6 weeks            5021
7-12 months          4015
Less than 1 week      729
Name: Age_Bucket, dtype: int64

In [121]:
# Drop Age and Age_upon_Outcome as information in Age_Bucket
df.drop(['Age', 'Age_upon_Outcome'], axis=1, inplace=True)

In [122]:
# Drop fixed_intake, fixed_outcome, fixed_change as information in Sex columns
df.drop(['fixed_intake', 'fixed_outcome', 'fixed_changed'], axis=1, inplace=True)

In [123]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71284 entries, 0 to 76976
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Animal ID           71284 non-null  object        
 1   Name_intake         71284 non-null  object        
 2   DateTime_intake     71284 non-null  datetime64[ns]
 3   Found_Location      71284 non-null  object        
 4   Intake_Type         71284 non-null  object        
 5   IntakeCondition     71284 non-null  object        
 6   Animal_Type_intake  71284 non-null  object        
 7   Sex_Intake          71284 non-null  object        
 8   Breed_intake        71284 non-null  object        
 9   Color_intake        71284 non-null  object        
 10  DateTime_outcome    71284 non-null  datetime64[ns]
 11  Outcome_Type        71284 non-null  object        
 12  Sex_upon_Outcome    71284 non-null  object        
 13  Age_Bucket          71284 non-null  object    

### Need to perform regex on
- Found_Location
- Breed_intake
- Color_intake
